In [1]:
import requests 
import bs4
from bs4 import BeautifulSoup
import csv 

In [2]:
movies_url = "https://elcinema.com/en/index/work/country/eg?page="
movie_url = "https://elcinema.com/en/work/"
#r = requests.get(URL)  ##get the data of the URL 

In [3]:
movies=[]
movieGenre=[]

In [ ]:
#soup = BeautifulSoup(r.content, 'html5lib') 

In [4]:
for i in range(1,503): # for the 502 pages
    URL=movies_url+str(i)
    print(URL)
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html5lib') 
    table = soup.findAll('table')[0]
    #table
    for row in table.findAll('tr'):
        record={}
        columns=row.findAll('td')
        if len(columns) > 0:
            #print(len(columns))
            #for col in columns:
            record['type']=columns[2].get_text()
            record['Release Year']=columns[3].get_text()
            if (record['type']== 'Movie' and abs(int(record['Release Year'])-2021) <=10):
                s = columns[1].findAll('a')[0].get('href')
                index = s.find('work' , 0 , len(s)-1)
                record['id']= s[index+5:-1]
                record['name']=columns[1].findAll('a')[1].get_text()
                record['Rating']=columns[4].findAll('span')[0].get_text()
                record['poster']=columns[1].findAll('img')[0].get('data-src')
                URL=movie_url+record['id']
                r = requests.get(URL)
                soup = BeautifulSoup(r.content, 'html5lib')
                genre=[]
                genreRecord= {}
                cat = soup.find('ul' , attrs={'id':"jump-here-genre"})
                if(type(cat)== bs4.element.Tag):
                    genres= cat.findAll('li')
                    for i in range(1, len(genres)):
                        genre.append(genres[i].a.get_text())
                    genreRecord['genre']=genre
                else:
                    genreRecord['genre']=['NONE']
                genreRecord['movieID']=record['id']
                desc = soup.find('p').get_text().replace('...Read more','')
                if(desc != "\n         Add a new topic\n      "):
                    record['description']= desc
                else:
                    record['description']='NONE'
                dateSoup = soup.find('ul' , attrs={'class':'list-separator inline'})
                if (type(dateSoup)== bs4.element.Tag):
                    releaseDate = dateSoup.find('li').findAll('a')[0].get_text()
                    releaseDate += ' ' + record['Release Year']
                    if (soup.find('ul' , attrs={'class':'list-separator inline'}).find('li').findAll('a')[0].get_text()[1].isdigit()):
                        record['release date']= releaseDate
                    else:
                        record['release date']= 'NONE'
                else:
                    record['release date']= 'NONE'
                    
                duration = soup.find('ul' , attrs={'class':'list-separator'}).findAll('li')[2].get_text()
                if(duration[0].isdigit()):
                     record['duration']= duration
                else:
                    record['duration']= 'NONE'
                
                ageRatingSoup = soup.find('ul' , attrs = { 'class' : 'censorship'})
                if((type(ageRatingSoup)== bs4.element.Tag)):
                    ageRating = ageRatingSoup.findAll('li')[1].get_text()
                    record['ageRating']=ageRating
                else:
                    record['ageRating']='NONE'
                    
                revenueURL = URL + '/boxoffice'
                #revenueURL = "https://elcinema.com/en/work/2066560/boxoffice"
                r = requests.get(revenueURL)
                revenueSoup = BeautifulSoup(r.content, 'html5lib') 
                revenue = revenueSoup('strong')[0]
                if ((type(revenue)== bs4.element.Tag)):
                    revenue = revenue.get_text()
                    record['Revenue']=revenue
                
                
                CastURL= URL + '/cast'
                #print(CastURL)
                r = requests.get(CastURL)
                castSoup = BeautifulSoup(r.content, 'html5lib') 
                castCat = castSoup.findAll('h3' , attrs= {'class':'section-title'})
                movieCast = []
                counts = [0]
                for i in range (0 , len(castCat)):
                    count = int(castCat[i].find('span').get_text().replace('(','').replace(')',''))
                    counts.append(count+counts[i])
                    role = castCat[i].find(text=True).replace('\n','').replace(' ','')
                    roleCast = castSoup.findAll('ul', attrs={'class':'description'})
                    for j in range (counts[i] ,counts[i]+count): 
                        castMemberDict={}
                        roleCastMember = roleCast[j].find('a').get_text()
                        roleCastMemberID=roleCast[j].find('a').get('href')
                        index = roleCastMemberID.find('person' , 0 , len(roleCastMemberID)-1)
                        roleCastMemberID= roleCastMemberID[index+7:-1]
                        #print(roleCastMember + ' ' +role+ ' '+roleCastMemberID)   
                        castMemberDict['name']=roleCastMember
                        castMemberDict['role']=role
                        castMemberDict['castID']=roleCastMemberID
                        movieCast.append(castMemberDict)
                
                
                record['cast']=movieCast
                
                releasedURL = URL + '/released'
                r = requests.get(releasedURL)
                premiereSoup= BeautifulSoup(r.content, 'html5lib')
                if (type(premiereSoup.find('table'))== bs4.element.Tag):
                    premiereDate = premiereSoup.find('table').findAll('tr')[1].findAll('td')[2].get_text()
                    record['premiereDate']= premiereDate
                else:
                    record['premiereDate']= 'NONE'

                #print(URL)
                #print(record)
                #print(genreRecord)
                value = record['id']
                list_of_bool = [True for elem in movies if value in elem.values()]
                if any(list_of_bool):
                    print('skipped')
                    #print(record)
                    continue #avoid adding movies which already exist in the table 
                else:
                    movies.append(record)
                    movieGenre.append(genreRecord)



https://elcinema.com/en/index/work/country/eg?page=1
https://elcinema.com/en/index/work/country/eg?page=2
https://elcinema.com/en/index/work/country/eg?page=3
https://elcinema.com/en/index/work/country/eg?page=4
https://elcinema.com/en/index/work/country/eg?page=5
https://elcinema.com/en/index/work/country/eg?page=6
https://elcinema.com/en/index/work/country/eg?page=7
https://elcinema.com/en/index/work/country/eg?page=8
https://elcinema.com/en/index/work/country/eg?page=9
https://elcinema.com/en/index/work/country/eg?page=10
https://elcinema.com/en/index/work/country/eg?page=11
skipped
skipped
https://elcinema.com/en/index/work/country/eg?page=12
https://elcinema.com/en/index/work/country/eg?page=13
https://elcinema.com/en/index/work/country/eg?page=14
https://elcinema.com/en/index/work/country/eg?page=15
https://elcinema.com/en/index/work/country/eg?page=16
https://elcinema.com/en/index/work/country/eg?page=17
https://elcinema.com/en/index/work/country/eg?page=18
https://elcinema.com/

https://elcinema.com/en/index/work/country/eg?page=149
https://elcinema.com/en/index/work/country/eg?page=150
https://elcinema.com/en/index/work/country/eg?page=151
https://elcinema.com/en/index/work/country/eg?page=152
https://elcinema.com/en/index/work/country/eg?page=153
https://elcinema.com/en/index/work/country/eg?page=154
https://elcinema.com/en/index/work/country/eg?page=155
https://elcinema.com/en/index/work/country/eg?page=156
https://elcinema.com/en/index/work/country/eg?page=157
https://elcinema.com/en/index/work/country/eg?page=158
https://elcinema.com/en/index/work/country/eg?page=159
https://elcinema.com/en/index/work/country/eg?page=160
https://elcinema.com/en/index/work/country/eg?page=161
https://elcinema.com/en/index/work/country/eg?page=162
https://elcinema.com/en/index/work/country/eg?page=163
https://elcinema.com/en/index/work/country/eg?page=164
https://elcinema.com/en/index/work/country/eg?page=165
https://elcinema.com/en/index/work/country/eg?page=166
https://el

https://elcinema.com/en/index/work/country/eg?page=298
https://elcinema.com/en/index/work/country/eg?page=299
https://elcinema.com/en/index/work/country/eg?page=300
https://elcinema.com/en/index/work/country/eg?page=301
https://elcinema.com/en/index/work/country/eg?page=302
https://elcinema.com/en/index/work/country/eg?page=303
https://elcinema.com/en/index/work/country/eg?page=304
https://elcinema.com/en/index/work/country/eg?page=305
https://elcinema.com/en/index/work/country/eg?page=306
https://elcinema.com/en/index/work/country/eg?page=307
https://elcinema.com/en/index/work/country/eg?page=308
https://elcinema.com/en/index/work/country/eg?page=309
https://elcinema.com/en/index/work/country/eg?page=310
https://elcinema.com/en/index/work/country/eg?page=311
https://elcinema.com/en/index/work/country/eg?page=312
https://elcinema.com/en/index/work/country/eg?page=313
https://elcinema.com/en/index/work/country/eg?page=314
https://elcinema.com/en/index/work/country/eg?page=315
https://el

https://elcinema.com/en/index/work/country/eg?page=447
https://elcinema.com/en/index/work/country/eg?page=448
https://elcinema.com/en/index/work/country/eg?page=449
https://elcinema.com/en/index/work/country/eg?page=450
https://elcinema.com/en/index/work/country/eg?page=451
https://elcinema.com/en/index/work/country/eg?page=452
https://elcinema.com/en/index/work/country/eg?page=453
https://elcinema.com/en/index/work/country/eg?page=454
https://elcinema.com/en/index/work/country/eg?page=455
https://elcinema.com/en/index/work/country/eg?page=456
https://elcinema.com/en/index/work/country/eg?page=457
https://elcinema.com/en/index/work/country/eg?page=458
https://elcinema.com/en/index/work/country/eg?page=459
https://elcinema.com/en/index/work/country/eg?page=460
https://elcinema.com/en/index/work/country/eg?page=461
https://elcinema.com/en/index/work/country/eg?page=462
https://elcinema.com/en/index/work/country/eg?page=463
https://elcinema.com/en/index/work/country/eg?page=464
https://el

In [5]:
print(len(movies))

625


In [6]:
movies

[{'type': 'Movie',
  'Release Year': '2023',
  'id': '2059336',
  'name': 'El Feel El Azraq 3',
  'Rating': ' 0 ',
  'poster': 'https://media.elcinema.com/blank_photos/75x75.jpg',
  'description': 'NONE',
  'release date': 'NONE',
  'duration': 'NONE',
  'ageRating': 'NONE',
  'Revenue': ' 0 EGP',
  'cast': [{'name': 'Karim Abdulaziz', 'role': 'Actor', 'castID': '1001574'},
   {'name': 'Nelly Karim', 'role': 'Actor', 'castID': '1013725'},
   {'name': 'Hend Sabry', 'role': 'Actor', 'castID': '1082742'},
   {'name': 'Marwan Hamed', 'role': 'Director', 'castID': '1068590'},
   {'name': 'Ahmed Mourad', 'role': 'Writer', 'castID': '2012950'}],
  'premiereDate': 'NONE'},
 {'type': 'Movie',
  'Release Year': '2022',
  'id': '2059875',
  'name': 'دماغ عالية',
  'Rating': ' 0 ',
  'poster': 'https://media.elcinema.com/uploads/_75x75_3d90d1ee22c5f455bc4556073eab69cd218446d6134dc0f2694782ee39ccb5bf.jpg',
  'description': 'NONE',
  'release date': 'NONE',
  'duration': 'NONE',
  'ageRating': 'NONE

In [7]:
uniqueMovies = []
for movie in movies:
    value = movie['id']
    list_of_bool = [True for elem in uniqueMovies if value in elem.values()]
    if any(list_of_bool):
        print('skipped')
        continue #avoid adding movies which already exist in the table 
    else:
        uniqueMovies.append(movie)

In [8]:
print(len(uniqueMovies))

625


In [9]:
uniqueGenres = []
for genre in movieGenre:
    value = genre['movieID']
    list_of_bool = [True for elem in uniqueGenres if value in elem.values() ]
    if any(list_of_bool):
        print('skipped')
        continue #avoid adding movies which already exist in the table 
    else:
        uniqueGenres.append(genre)
    #movieGenre.append(genreRecord)

In [10]:
print(len(uniqueGenres))

625


In [ ]:
 '''
URL="https://elcinema.com/en/work/2066560/boxoffice"
print(URL)
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib') 
revenue = soup('strong')[0].get_text()
print(revenue)
'''

In [ ]:
'''
URL="https://elcinema.com/en/work/2003980/cast"
print(URL)
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib') 
castCat = soup.findAll('h3' , attrs= {'class':'section-title'})
movieCast = []
counts = [0]
for i in range (0 , len(castCat)):
    count = int(castCat[i].find('span').get_text().replace('(','').replace(')',''))
    print(count)
    counts.append(count+counts[i])
    role = castCat[i].find(text=True).replace('\n','').replace(' ','')
    roleCast = soup.findAll('ul', attrs={'class':'description'})
    for j in range (counts[i] ,counts[i]+count): 
        castMemberDict={}
        roleCastMember = roleCast[j].find('a').get_text()
        roleCastMemberID=roleCast[j].find('a').get('href')
        index = roleCastMemberID.find('person' , 0 , len(roleCastMemberID)-1)
        roleCastMemberID= roleCastMemberID[index+7:-1]
        print(roleCastMember + ' ' +role+ ' '+roleCastMemberID)   
        castMemberDict['name']=roleCastMember
        castMemberDict['role']=role
        castMemberDict['castID']=roleCastMemberID
        movieCast.append(castMemberDict)
    
#print(movieCast)
'''

In [13]:
#processing the cast Table 
#CastMembers= []
castURLconst = "https://elcinema.com/en/person/"
for movie in movies:
    cast = movie['cast']
    for member in cast:
        castMember={}
        #if(not(member in CastMembers)):
        value = member['castID']
        list_of_bool = [True for elem in CastMembers if value in elem.values()]
        if any(list_of_bool):
            print('skipped')
            print(member['name'])
            continue #avoid adding cast members who already exists in the table 
        else:
            castMember['ID']=member['castID']
            castMember['name']=member['name']
            castURL = castURLconst + member['castID']
            print(castURL)
            r = requests.get(castURL)
            soup = BeautifulSoup(r.content, 'html5lib') 
            nationalitySoup = soup.findAll('ul' , attrs ={'class':"list-separator list-title"})
            if (len(nationalitySoup)>0):
                nationalityS = soup.findAll('ul' , attrs ={'class':"list-separator list-title"})[0].find('a')
                if(type(nationalityS)==bs4.element.Tag):
                    nationality = nationalityS.get_text()
                #print(nationality)
                    castMember['Nationality']=nationality
                else:
                    castMember['Nationality']='NONE'
                    
                #print(len(nationalityS))
                if(len(nationalitySoup) > 1) :
                    DOBSoup = soup.findAll('ul' , attrs ={'class':"list-separator list-title"})[1].findAll('a')
                    if (len(DOBSoup) == 2): # day / Year
                        DOB = DOBSoup[0].get_text()+ ' '+ DOBSoup[1].get_text()
                        castMember['DOB']=DOB
                        #print(DOB)
                    else:
                        castMember['DOB']='NONE'
                else:
                    castMember['DOB']='NONE'
                if(len(nationalitySoup)>2):
                    bcSoup = soup.findAll('ul' , attrs ={'class':"list-separator list-title"})[2].findAll('li')
                    if(len(bcSoup) > 0):
                        birthCountry = bcSoup[1].get_text()
                        castMember['BirthCountry']=birthCountry
                    else:
                        castMember['BirthCountry']='NONE'
                else:
                    castMember['BirthCountry']='NONE'
                
                
            else:
                castMember['Nationality']='NONE'
                castMember['DOB']='NONE'
                castMember['BirthCountry']='NONE'
                
            if(soup.find('div' , attrs = {'class': 'columns medium-3 large-3'}).find('img')):
                image = soup.find('div' , attrs = {'class': 'columns medium-3 large-3'}).find('img').get('src')
            #print(image)
                castMember['image']=image
            else:
                castMember['image']='NONE'
            biographySoup = soup.find('p' , attrs = {'class': 'no-margin'})
            if(type(biographySoup)==bs4.element.Tag):
                biography=biographySoup.get_text().replace('...Read more','')
                castMember['biography']=biography
            else:
                castMember['biography']='NONE'
            
                
            #if(DOB[0].isdigit()):
                #print(DOB)
            
                    
            #print(castMember)
            CastMembers.append(castMember)
#print(len(CastMembers))
#print(CastMembers)

skipped
Karim Abdulaziz
skipped
Nelly Karim
skipped
Hend Sabry
skipped
Marwan Hamed
skipped
Ahmed Mourad
skipped
Ahmad Fahmy
skipped
Ruby
skipped
Mohamed Tharwat
skipped
Bayyumy Fouad
skipped
Mostafa Khater
skipped
Amr Abdul Gelil
skipped
Mayan El Sayed
skipped
Akram Hosny
skipped
Karim Abdulaziz
skipped
Synergy Art Production
skipped
Tamer Mursy
skipped
Ihab Blibl
skipped
Ahmad Fahmy
skipped
Ahmed El Gendy
skipped
Amr Assem
skipped
Refaat Abdulhakeem
skipped
Karim Abdulaziz
skipped
Ahmed Ezz
skipped
Hend Sabry
skipped
Ruby
skipped
Ahmed Malek
skipped
Amina Khalil
skipped
Asmaa Abou El Yazeed
skipped
Arefa Abdul Rasoul
skipped
Huda El Mufti
skipped
Yassin Amir
skipped
Aly Kassem
skipped
Sayed Ragab
skipped
Eyad Nassar
skipped
Ahmed Kamal
skipped
Synergy Art Production
skipped
Tamer Mursy
skipped
Marwan Hamed
skipped
Ahmed Mourad
skipped
Asser Yassine
skipped
Ashraf El Ashmawi
skipped
Khaled El Sawy
skipped
Shery Adel
skipped
Mohamed Sallam
skipped
Esam Al-Saqqa
skipped
Yasser El Tobgy


skipped
Injy Al Moqaddem
skipped
Ahmed Anan (Ahmed Khaled)
skipped
Ibrahim Ghareib
skipped
Salma Tarek
skipped
Omar Sharqi
skipped
Basma Nabil
skipped
Dina Al Maghraby
skipped
Omar Ezz Eldein
skipped
Sabour Abo Eldahab
skipped
Haitham Soliman
skipped
Ahmed Abdelwahab
skipped
Mahmoud Naser
skipped
Mohamed Elhalaby
skipped
Joseph Louis Mokadem
skipped
Nader Wadea
skipped
محمد حامد
skipped
Ahmed Bigo
skipped
Kareem Elyamany
skipped
Eslam Rozza
skipped
Omar Moustafa
skipped
Islam Fadel
skipped
Hisham Watany
skipped
Maryam Saleh
skipped
Eslam Rozza
skipped
Rima Kotb
skipped
Camba
skipped
Gamal Nagib
skipped
Haifa Wehbe
skipped
Ahmed El Fishawy
skipped
Mostafa Khater
skipped
Arwa Gouda
skipped
Bayyumy Fouad
skipped
Abbas Abo El Hassan
skipped
Bassem Samra
skipped
Saad El Moukhtar
skipped
Karim Farouk
skipped
Amin  Gamal
skipped
Muhammad Abu Al-Saad
skipped
Sherif Yousry
skipped
Ahmad Mazhar
skipped
Waleed El Sabagh
skipped
Ahmed Tammam
skipped
mohamad wazeri
skipped
Synergy Art Production
sk

skipped
Samy Maghawry
skipped
Farida Saif Al-Nasr
skipped
Mohamed El Sawy
skipped
Ahmad Fahmy
skipped
Mostafa Khater
skipped
Osama Abdullah
skipped
Safaa Jalal
skipped
Abdel Aziz Hussein
skipped
Ahmad Mazhar
skipped
Chico
skipped
Mohamed Abdul Rahman
skipped
Hamdy Al-Merghany
skipped
Tamer Abdul Hamid
skipped
Omar Sami
skipped
Ahmed Amer
skipped
Yasser Howidy
skipped
Amr Elseyoufi
skipped
Synergy Art Production
skipped
Tamer Mursy
skipped
Wafaa Amer
skipped
Nedal Al-Shafeay
skipped
Mahmoud Al-Bezzawy
skipped
Ahmed Kamal
skipped
Ali Al Shujairi
skipped
Abir Monir
skipped
Zahra el haroufi
skipped
Hossam Elmedany
skipped
Kareem Fathi
skipped
mohamed hesham amer
skipped
Awad Maher
skipped
Evan Adeeb
skipped
محمد عباس صابر
skipped
SeifEl-din Fakhry
skipped
Azmy El rahmany
skipped
Wessam Al Madany
skipped
Wessam Al Madany
skipped
Mohamed Nassef
skipped
منى الزرقاني
skipped
رحمة باللافي
skipped
Hosny arabi
skipped
Manal Awad	
skipped
Maisa Abd Elhadi
skipped
Ali Suliman
skipped
Hany Abu-Assad

skipped
رغدة عصام حسني
skipped
Rania El-Montaser bellah
skipped
بسنت الهواري
skipped
Ramy Darrag
skipped
Ibrahim Issa
skipped
Mohamed Abd Elghany
skipped
Khaled Al Kammar
skipped
سامح سليم
skipped
Monia Fath El Bab
skipped
Karim El Sobky
skipped
Omar Taher
skipped
Islam Khairy
skipped
Mohamed Henidy
skipped
Walid Sabry
skipped
Elham Shahein
skipped
Amina Khalil
skipped
Kamel Elbasha
skipped
Ahmed Magdy
skipped
Ahmed Hatem
skipped
Mahmoud El-Lisy
skipped
Arefa Abdul Rasoul
skipped
Muhra Medhat
skipped
S Films Production & Media Services
skipped
Moataz Abdelwahab
skipped
Safie Eddeen Mahmoud
skipped
Ali Fayez
skipped
Heba Othman
skipped
Mohamed Mamdouh Ibrahem
skipped
Amir Ramses
skipped
Amir Ramses
skipped
إيمان العلبي
skipped
Mahmoud fouda
skipped
Nahed Nasrallah
skipped
Tamer Karawan
skipped
Hanan Motawee
skipped
Rania Shaheen
skipped
Fatima Adel
skipped
Amr Gamal
skipped
Khaled Khattab
skipped
Assem Nagaty
skipped
Hanan Youssef
skipped
Mohamed Radwan
skipped
Kareem El-Attar
skipped
N

skipped
Lubna Mahmoud
skipped
Bishoy Taher
skipped
Noha Adel
skipped
Abdel Moneim Riyadh
skipped
Bahgat Adly
skipped
Shorouq
skipped
Heidi Sleim
skipped
George Ghabour
skipped
Tarek Waly
skipped
Reda Ahmed
skipped
Mahmoud Mazika
skipped
Mahmoud Jamaica
skipped
Ashraf Hosny
skipped
Kareem Fathi
skipped
Hala Almadani
skipped
Mamdouh Al Shennawi
skipped
Amira Elsherif
skipped
Samar Gaber
skipped
Rania Mansour
skipped
Heba Magdy
skipped
Nour Mahmoud
skipped
Rania Yousef
skipped
Ahmed Seleem
skipped
سيف هنو
skipped
Rania El Beshbishy
skipped
Reda Abdel Razek
skipped
Reda Abdel Razek
skipped
Akram Abd El Aziz
skipped
Frederick L. Greene
skipped
Marco Orsini
skipped
Omar Nour	
skipped
Omar Samra	
skipped
Marco Orsini
skipped
Mustafa Shehata	
skipped
Islam Mubarak	
skipped
Moatasem Rashed	
skipped
Bonna Khalid	
skipped
Talal Afifi	
skipped
Mahmoud Alsarraj	
skipped
Mazen Ahmed
skipped
Amal Mustafa
skipped
Hassan Ali
skipped
Mohamed Khalil
skipped
Asjad Mohamed
skipped
Yousef Ibrahim
skipped
Am

...

https://elcinema.com/en/person/2008111
https://elcinema.com/en/person/2010829
https://elcinema.com/en/person/1106958
https://elcinema.com/en/person/2010828
skipped
Sharif Izz al-Din
skipped
Sama Al Massri
https://elcinema.com/en/person/2009149
skipped
Walid Farouk
https://elcinema.com/en/person/2009624
skipped
Ehab Hegazi
skipped
Ehab Hegazi
https://elcinema.com/en/person/2009179
skipped
Safwan Nasser El Din
skipped
Ramy Rizkallah
skipped
Bushra
skipped
Walid Farouk
skipped
Walid Farouk
skipped
Mina Nabil
https://elcinema.com/en/person/2008605
skipped
Mina Nabil
skipped
محمد عبد الرؤوف
skipped
Abdelsalam Moussa
skipped
Omar Khaled
https://elcinema.com/en/person/1100089
skipped
Emmy
skipped
Mohamed Ramadan
skipped
Romany Saad
skipped
Romany Saad
skipped
Marouan Omara
https://elcinema.com/en/person/1016144
skipped
Rania Yousef
skipped
Yasmin Raeis
skipped
Amr Youssef
skipped
Basma
skipped
Kinda Alloush
skipped
Zizi El Badrawi
skipped
Hanan Suliman
skipped
Mai Fakhry
skipped
Zaki Fateen A

skipped
محمد الشحات
skipped
Mohammad Adel
skipped
Amr Ossama
skipped
Lala Al Sisi
skipped
Muhammad Ashraf
https://elcinema.com/en/person/1111396
https://elcinema.com/en/person/2013127
https://elcinema.com/en/person/2020923
https://elcinema.com/en/person/2011977
https://elcinema.com/en/person/2020924
https://elcinema.com/en/person/2020925
https://elcinema.com/en/person/2020926
https://elcinema.com/en/person/1994896
https://elcinema.com/en/person/1995731
https://elcinema.com/en/person/2120924
skipped
Khaled Youssef
https://elcinema.com/en/person/1100479
skipped
Eslam Wifky
https://elcinema.com/en/person/1110770
skipped
Ahmad Afifi
https://elcinema.com/en/person/2119828
skipped
Yasmine Ahmed Kamel
skipped
Nasser Abdulrahman
skipped
Khaled Youssef
skipped
Gamal Bakhit
https://elcinema.com/en/person/1990689
https://elcinema.com/en/person/1100481
https://elcinema.com/en/person/1101755
skipped
Ouf Abd Elrahaman
skipped
غادة عز الدين
https://elcinema.com/en/person/1101916
skipped
Islam Gouda
s

skipped
Ahmed Fouad Selim
skipped
Sameh Abu Al-Ghar
skipped
Abdel Salaam El Dahshan
skipped
Hazem Samir
skipped
Ahmad Haiba
skipped
Maggie Adel
https://elcinema.com/en/person/1002914
skipped
Sohaila Hussein
skipped
وسام هنداوي
skipped
Entisar Hassan
skipped
Maryam Saeed Saleh
https://elcinema.com/en/person/1109102
skipped
لالا ياسر
skipped
Magdy Shokry
skipped
Muhja Abdulrahman
skipped
Heba Samir
skipped
Maged Abdul Azim
https://elcinema.com/en/person/1995536
https://elcinema.com/en/person/1995530
skipped
المجموعة الفنية المتحدة
skipped
sherif shehata
skipped
Othman Abu Laban
skipped
Ahmed Hussein
skipped
Hany Ramzy
skipped
Dorra
skipped
Edward
https://elcinema.com/en/person/2019751
skipped
Youssef Fawzy
skipped
Tatiana
skipped
Genna Amr
skipped
Diaa El Merghany
skipped
Ahmed Halawa
https://elcinema.com/en/person/1025740
https://elcinema.com/en/person/2019748
https://elcinema.com/en/person/2019750
https://elcinema.com/en/person/2019749
skipped
Muhammad Salah
https://elcinema.com/en/per

https://elcinema.com/en/person/1026152
skipped
Soliman Eid
skipped
Samar Gaber
skipped
Jamal Hejazi
https://elcinema.com/en/person/1983281
skipped
Tarek Essam
skipped
Youssef Dawood
https://elcinema.com/en/person/2129683
skipped
Wael Jassar
skipped
Said El Marouk
https://elcinema.com/en/person/2096148
skipped
Dina Mohammed Karim
skipped
Karim elmehy
https://elcinema.com/en/person/2011549
skipped
Magdi Seiko
skipped
amer magdy
skipped
عدنان شريف
skipped
Mohamed shakal
skipped
سامر الجمال
https://elcinema.com/en/person/1689919
skipped
islam elmaghawry
skipped
Refaat Abdulhakeem
skipped
Maya Haddad
skipped
Youssef Maaty
skipped
Attia Amin
skipped
وائل درويش
skipped
Amr Ismail
skipped
Ahmed Hafez
skipped
mohamed el salamony
skipped
Sawsan Bader
skipped
Ruby
skipped
Maryhan
skipped
Ahmed Azmy
skipped
Mohamed Ramadan
skipped
Ibrahim El-Nagary
skipped
Doaa Teama
skipped
Boutros Ghaly
skipped
Salwa Mohamed Ali
skipped
Youssef Ismail
skipped
Karim Al-Abnoudy
skipped
Maher Selim
skipped
Arefa Ab

skipped
Fekry Sadek
skipped
Mohamed El Keblawy
https://elcinema.com/en/person/1110772
https://elcinema.com/en/person/1982648
https://elcinema.com/en/person/1990686
https://elcinema.com/en/person/1008612
skipped
Sayed Seif
https://elcinema.com/en/person/1982921
https://elcinema.com/en/person/2009696
skipped
Ahmed Darwish
https://elcinema.com/en/person/1100039
skipped
DayDream Art Production
skipped
Omar Elzohary
skipped
Omar Elzohary
https://elcinema.com/en/person/2007144
skipped
Tamer Mohsen
skipped
Tamer Mohsen
https://elcinema.com/en/person/1983708
skipped
Hanan Rady
https://elcinema.com/en/person/2009680
https://elcinema.com/en/person/2010382
https://elcinema.com/en/person/2014222
https://elcinema.com/en/person/2014223
https://elcinema.com/en/person/2014224
https://elcinema.com/en/person/2014221
https://elcinema.com/en/person/2014220
skipped
Viola Shafik
https://elcinema.com/en/person/2014225
skipped
Viola Shafik
skipped
bassam elelaimy
https://elcinema.com/en/person/1033218
skipped

In [14]:
print(len(CastMembers))
print(CastMembers[0])

6984
{'ID': '1001574', 'name': 'Karim Abdulaziz', 'Nationality': 'Egypt', 'DOB': '17 August 1975', 'BirthCountry': 'Egypt', 'image': 'https://media.elcinema.com/uploads/_315x420_2b2a9c81d61f9b446bf6ff1232b86e7c4f5fae96d3eae40faff0eccdeba3b391.jpg', 'biography': 'Son of famous director Mohamed Abdel Aziz, Karim was no stranger to the world of film making. He graduated from the institute of high arts, directing division, in 1997. He worked as an assistant director for a short time before deciding to jump in front of the camera. Director Sherif Emara gave him his first role, but Karim\'s notoriety grew after his TV role in the series "Lady from the Era of Love" starring Samira Ahmed. Following the show, he was given his first lead role on the silver screen in the movie "Why did you Make me Love You?" opposite Mona Zaki and Hala Sheeha.'}


In [15]:
uniqueCast = []
for member in CastMembers:
    value = member['ID']
    list_of_bool = [True for elem in uniqueCast if value in elem.values()]
    if any(list_of_bool):
        print('skipped')
        continue #avoid adding movies which already exist in the table 
    else:
        uniqueCast.append(member)

In [16]:
print(len(uniqueCast))

6984


In [17]:
#writing movieGenre table
filename = 'movieGenre.csv'
with open(filename, 'w', newline='') as f: 
    w = csv.writer(f) 
    w.writerow(["movieID","genre"])
    for line in movieGenre:
        #print(line)
        #w.writerow([line['movieID']])
        #w.writerow
        for genre in line['genre']:
            #print(genre)
            w.writerow([line['movieID'],genre])

In [18]:
casst = list(CastMembers)
for member in casst:
    if(member['DOB'][0].isspace()):
        member['DOB']=member['DOB'][1:]
        
        

In [19]:
#writing castMembers Table 
filename = 'castMembers.csv'
with open(filename, 'w', newline='' , encoding="utf-8-sig" ) as f: 
    w = csv.DictWriter(f,['ID','name','Nationality','DOB','BirthCountry','image','biography']) 
    w.writeheader() 
    for line in casst: 
        w.writerow(line) 

In [20]:
len(movieCast)

5

In [21]:
moviess = movies.copy()
for moviie in moviess:
    if(moviie['release date'][0].isspace()):
        moviie['release date']=moviie['release date'][1:]

In [22]:
for movie in moviess:
    movie['Revenue']=movie['Revenue'].replace(' EGP','')
    movie['Revenue']=movie['Revenue'].replace(',','')
    movie['Rating']=movie['Rating'].replace(' ','')
    if(movie['Revenue'][0].isspace()):
        movie['Revenue']=movie['Revenue'][1:]
    

In [23]:
#writing Movies Table 
filename = 'Movies.csv'
with open(filename, 'w', newline='' , encoding="utf-8-sig" ) as f: 
    w = csv.DictWriter(f,['id','description','premiereDate','type','Release Year','name','Rating','poster','release date','duration','ageRating','Revenue'],extrasaction='ignore') 
    w.writeheader() 
    for line in moviess: 
        w.writerow(line) 

In [24]:
#writing movieCast table
filename = 'movieCast.csv'
with open(filename, 'w', newline='') as f: 
    w = csv.writer(f) 
    w.writerow(["movieID","CastID","Role"])
    for movie in movies:
        #print(line)
        #w.writerow([line['movieID']])
        #w.writerow
        for cast in movie['cast']:
            #print(genre)
            w.writerow([movie['id'], cast['castID'], cast['role']])

# Scrape to MySql

In [ ]:
!pip install pymysql

In [25]:
import pymysql.cursors   

In [47]:
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='medo1236987450',
                             db='egyptianimdb',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [27]:
try:
    with connection.cursor() as cursor:
        for movie in movies:
            name = movie['name'] if movie['name'] != 'NONE' else None
            movie_id = movie['id'] if movie['id'] != 'NONE' else None
            releaseDate = movie['release date'] if movie['release date'] != 'NONE' else None
            description = movie['description'] if movie['description'] != 'NONE' else None 
            duration = movie['duration'] if movie['duration'] != 'NONE' else None 
            revenue = int(movie['Revenue']) if movie['Revenue'] != 'NONE' else None
            ageRating = movie['ageRating'] if movie['ageRating'] != 'NONE' else None
            premiereDate = movie['premiereDate']if movie['premiereDate'] != 'NONE' else None
            poster = movie['poster']if movie['poster'] != 'NONE' else None
            avg_rating = float(movie['Rating']) if movie['Rating'] != 'NONE' else None
            sql = "INSERT INTO `movie` (`MovieName`, `ID`, `ReleaseDate`, `MovieDescription`, `Duration`,`TotalRevenue`,`AgeRating`,`PremiereDate`,`poster`,`AVG_RATING`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, (name, movie_id,releaseDate , description, duration,revenue,ageRating,premiereDate,poster,avg_rating))
            
    connection.commit()
finally:
    connection.close()

In [29]:
try:
    with connection.cursor() as cursor:
        for member in casst :
            sql = "INSERT INTO `castmember` (`FullName`, `ID`, `Nationality`, `BirthCountry`, `Biography`,`date_Of_Birth`,`image`) VALUES (%s, %s,%s,%s,%s,%s,%s)"
            cursor.execute(sql, (member['name'], member['ID'], member['Nationality'], member['BirthCountry'], member['biography'],member['DOB'],member['image']))
            
    connection.commit()
finally:
    connection.close()

In [43]:
a_csv_file = open("movieGenre.csv", "r")
dict_reader = csv. DictReader(a_csv_file)
movie_genre_dict = list(dict_reader)
#movie_genre_dict = dict(ordered_dict_from_csv)
print(len(movie_genre_dict))
print(movie_genre_dict)


967
[{'movieID': '2059336', 'genre': 'Thriller'}, {'movieID': '2059336', 'genre': 'Horror'}, {'movieID': '2059875', 'genre': 'Comedy'}, {'movieID': '2058736', 'genre': 'Drama'}, {'movieID': '2058736', 'genre': 'History'}, {'movieID': '2068611', 'genre': 'Drama'}, {'movieID': '2068366', 'genre': 'Comedy'}, {'movieID': '2067595', 'genre': 'NONE'}, {'movieID': '2067574', 'genre': 'Action'}, {'movieID': '2067574', 'genre': 'Crime'}, {'movieID': '2067574', 'genre': 'Thriller'}, {'movieID': '2067020', 'genre': 'NONE'}, {'movieID': '2067013', 'genre': 'Drama'}, {'movieID': '2067013', 'genre': 'War'}, {'movieID': '2066560', 'genre': 'Comedy'}, {'movieID': '2065450', 'genre': 'NONE'}, {'movieID': '2065401', 'genre': 'Romance'}, {'movieID': '2065401', 'genre': 'Comedy'}, {'movieID': '2065225', 'genre': 'NONE'}, {'movieID': '2065010', 'genre': 'NONE'}, {'movieID': '2065004', 'genre': 'Drama'}, {'movieID': '2064895', 'genre': 'NONE'}, {'movieID': '2064843', 'genre': 'NONE'}, {'movieID': '2064814',

In [45]:
try:
    with connection.cursor() as cursor:
        for genre in movie_genre_dict :
            genre_value = genre['movieID'] if  genre['movieID'] != 'NONE' else None 
            sql = "INSERT INTO `moviegenre` (`movieID`, `Genre`) VALUES (%s, %s)"
            cursor.execute(sql, (genre_value, genre['genre']))
            
    connection.commit()
finally:
    connection.close()

In [46]:
a_csv_file = open("movieCast.csv", "r")
dict_reader = csv. DictReader(a_csv_file)
movie_cast_dict = list(dict_reader)
#movie_genre_dict = dict(ordered_dict_from_csv)
print(len(movie_cast_dict))
print(movie_cast_dict)

17635
[{'movieID': '2059336', 'CastID': '1001574', 'Role': 'Actor'}, {'movieID': '2059336', 'CastID': '1013725', 'Role': 'Actor'}, {'movieID': '2059336', 'CastID': '1082742', 'Role': 'Actor'}, {'movieID': '2059336', 'CastID': '1068590', 'Role': 'Director'}, {'movieID': '2059336', 'CastID': '2012950', 'Role': 'Writer'}, {'movieID': '2059875', 'CastID': '1977444', 'Role': 'Actor'}, {'movieID': '2059875', 'CastID': '1005695', 'Role': 'Actor'}, {'movieID': '2059875', 'CastID': '1980725', 'Role': 'Actor'}, {'movieID': '2059875', 'CastID': '1110017', 'Role': 'Actor'}, {'movieID': '2059875', 'CastID': '2069218', 'Role': 'Actor'}, {'movieID': '2059875', 'CastID': '1035185', 'Role': 'Actor'}, {'movieID': '2059875', 'CastID': '2110145', 'Role': 'Actor'}, {'movieID': '2059875', 'CastID': '1110533', 'Role': 'Actor'}, {'movieID': '2059875', 'CastID': '1001574', 'Role': 'Actor'}, {'movieID': '2059875', 'CastID': '1983833', 'Role': 'Producer'}, {'movieID': '2059875', 'CastID': '1100347', 'Role': 'Pro

In [48]:
try:
    with connection.cursor() as cursor:
        for cast in movie_cast_dict :
            sql = "INSERT INTO `moviecast` (`movieID`, `castID` , `Role`) VALUES (%s, %s , %s)"
            #print(cast['CastID'])
            cursor.execute(sql, (cast['movieID'], cast['CastID'], cast['Role']))
            
    connection.commit()
finally:
    connection.close()